In [1]:
#[Problem 1] Execution of various methods
import tensorflow as tf
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, LSTM, GRU, Dense

max_features = 10000 
maxlen = 200        

(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test  = sequence.pad_sequences(x_test, maxlen=maxlen)

def build_model(rnn_layer):
    model = Sequential([
        Embedding(max_features, 128),
        rnn_layer,
        Dense(1, activation="sigmoid")
    ])
    model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
    return model

model_rnn = build_model(SimpleRNN(64))
model_rnn.fit(x_train, y_train, epochs=2, batch_size=64, validation_data=(x_test, y_test))

model_gru = build_model(GRU(64))
model_gru.fit(x_train, y_train, epochs=2, batch_size=64, validation_data=(x_test, y_test))

model_lstm = build_model(LSTM(64))
model_lstm.fit(x_train, y_train, epochs=2, batch_size=64, validation_data=(x_test, y_test))


17464789/17464789 ━━━━━━━━━━━━━━━━━━━━ 40s 2us/step 
Epoch 1/2
391/391 ━━━━━━━━━━━━━━━━━━━━ 33s 80ms/step - accuracy: 0.6227 - loss: 0.6187 - val_accuracy: 0.7771 - val_loss: 0.4737
Epoch 2/2
391/391 ━━━━━━━━━━━━━━━━━━━━ 32s 81ms/step - accuracy: 0.8596 - loss: 0.3358 - val_accuracy: 0.7074 - val_loss: 0.5675
Epoch 1/2
391/391 ━━━━━━━━━━━━━━━━━━━━ 70s 174ms/step - accuracy: 0.7024 - loss: 0.5307 - val_accuracy: 0.8569 - val_loss: 0.3344
Epoch 2/2
391/391 ━━━━━━━━━━━━━━━━━━━━ 67s 171ms/step - accuracy: 0.8971 - loss: 0.2611 - val_accuracy: 0.8742 - val_loss: 0.3033
Epoch 1/2
391/391 ━━━━━━━━━━━━━━━━━━━━ 70s 175ms/step - accuracy: 0.7224 - loss: 0.5264 - val_accuracy: 0.8519 - val_loss: 0.3444
Epoch 2/2
391/391 ━━━━━━━━━━━━━━━━━━━━ 74s 190ms/step - accuracy: 0.8942 - loss: 0.2706 - val_accuracy: 0.8617 - val_loss: 0.3156


In [3]:

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import ConvLSTM2D, BatchNormalization, Dense

model_conv = Sequential([
    ConvLSTM2D(filters=32, kernel_size=(3, 3),
               input_shape=(10, 64, 64, 1), # (time, height, width, channels)
               padding="same", return_sequences=True),
    BatchNormalization(),
    ConvLSTM2D(filters=32, kernel_size=(3, 3),
               padding="same", return_sequences=False),
    BatchNormalization(),
    Dense(10, activation="softmax")
])


In [4]:
#[Problem 2] Comparison on another dataset
from tensorflow.keras.datasets import reuters
from tensorflow.keras.preprocessing import sequence

max_words = 10000
maxlen = 200

(x_train, y_train), (x_test, y_test) = reuters.load_data(num_words=max_words)
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test  = sequence.pad_sequences(x_test, maxlen=maxlen)

model = Sequential([
    Embedding(max_words, 128),
    LSTM(64),
    Dense(46, activation="softmax")
])

model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
history = model.fit(x_train, y_train, epochs=3, batch_size=64, validation_data=(x_test, y_test))


2110848/2110848 ━━━━━━━━━━━━━━━━━━━━ 3s 1us/step 
Epoch 1/3
141/141 ━━━━━━━━━━━━━━━━━━━━ 19s 123ms/step - accuracy: 0.3887 - loss: 2.7486 - val_accuracy: 0.5178 - val_loss: 1.8184
Epoch 2/3
141/141 ━━━━━━━━━━━━━━━━━━━━ 18s 125ms/step - accuracy: 0.5198 - loss: 1.8254 - val_accuracy: 0.5574 - val_loss: 1.7602
Epoch 3/3
141/141 ━━━━━━━━━━━━━━━━━━━━ 18s 129ms/step - accuracy: 0.5518 - loss: 1.7108 - val_accuracy: 0.5686 - val_loss: 1.6950


**[Problem 3] Other related classes in Keras**

The class called RNN in Keras is a kind of general framework for recurrent networks. It does not specify whether the network is SimpleRNN, GRU, or LSTM. Instead, it is like a container that can run any type of recurrent “cell” step by step across a sequence. This is mostly used when you want to build a custom recurrent network from lower-level parts.

A SimpleRNNCell represents just one step of a SimpleRNN. When you use the full SimpleRNN layer, it is actually repeating this cell many times, once for each time step in the input sequence. You normally don’t use the cell directly, but it exists in case you want very fine control over what happens inside the network.

A GRUCell is the same idea, but for the GRU architecture. It represents a single GRU unit with its reset and update gates. A GRU layer is simply many of these GRUCells applied one after another through the sequence.

An LSTMCell is again similar, but for the LSTM. It represents one LSTM unit with its input, forget, and output gates, as well as its internal memory state. The LSTM layer is nothing more than a series of LSTMCells being applied step by step.

There is also StackedRNNCells, which lets you combine several different cells together so they act as one larger recurrent layer. For example, you could place an LSTMCell and a GRUCell together inside this stack, and the RNN wrapper would run them as a sequence. This feature gives flexibility for experimentation, but it is not very commonly used in real applications.

Finally, there are CuDNNGRU and CuDNNLSTM. These were special versions of GRU and LSTM designed to run much faster on NVIDIA GPUs by using the cuDNN library. They used to be important for performance, but in modern TensorFlow and Keras the standard GRU and LSTM layers automatically make use of GPU acceleration, so the CuDNN-specific classes are now mostly obsolete.
